In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s /content/drive/My\ Drive /mydrive
!ls /mydrive
!nvidia-smi

Mounted at /content/drive
'Colab Notebooks'   Documents	        speech-recognition
 data		    envs	        tilosite.gsite
 debug		    language-modeling   wandb.key
Wed Oct 14 08:07:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+---

In [2]:
%%time
%%bash
apt-get update && apt-get install -qq bc tree sox libsndfile1
rm -rf espnet
git clone -b tilos_minimal_librispeech https://github.com/dertilo/espnet.git
cd espnet && pip install -e .
pip install util@git+https://git@github.com/dertilo/util.git#egg=util


Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu18

Cloning into 'espnet'...
ERROR: tensorflow 2.3.0 has requirement h5py<2.11.0,>=2.10.0, but you'll have h5py 2.9.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Running command git clone -q 'https://****@github.com/dertilo/util.git' /tmp/pip-install-mpx7gbf6/util


CPU times: user 21.2 ms, sys: 9.83 ms, total: 31 ms
Wall time: 2min 4s


In [3]:
%%time
%%bash
ls -alth /mydrive/data
TARFILE=dev-clean_preprocessed.tar.gz
cp /mydrive/data/$TARFILE ./
tar xfz $TARFILE -C ./
ls -alth
du -sh ./dev-clean_preprocessed/



total 1.9G
-rw------- 1 root root 971M Sep 29 14:09 asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best.zip
-rw------- 1 root root  55M Sep 28 16:07 dev-clean_preprocessed.tar.gz
-rw------- 1 root root 636M Jun  7 08:35 decisions_185_StGB_no_content.jsonl.gz
-rw------- 1 root root 279M Jan  9  2020 BverfG_juris_content.jsonl.gz
total 56M
drwxr-xr-x  1 root root 4.0K Oct 14 08:10 .
-rw-------  1 root root  55M Oct 14 08:10 dev-clean_preprocessed.tar.gz
drwxr-xr-x 18 root root 4.0K Oct 14 08:08 espnet
drwxr-xr-x  1 root root 4.0K Oct 14 08:07 ..
drwx------  5 root root 4.0K Oct 14 08:07 drive
drwxr-xr-x  1 root root 4.0K Oct  5 16:31 .config
drwxr-xr-x  1 root root 4.0K Oct  5 16:31 sample_data
drwxrwxr-x  2 1000 1000 184K Sep 28 16:03 dev-clean_preprocessed
62M	./dev-clean_preprocessed/
CPU times: user 4.51 ms, sys: 1.89 ms, total: 6.4 ms
Wall time: 2.03 s


In [6]:
%%bash
pip install wandb


  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=645ffb0f7ea1e065a806e33ed3b4a6213d00f9bf14a315340e8ca8f18f107056
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=ef483908a98eb0508b479ded06e967691a77311973a07b392fbda3a87bd6d5bf
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=1dcc1e5237056fe5598afa74e50e67ca69010948630a06a68bebe880fecbe21b
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built watchdog subprocess32 pathtools


In [4]:
%%time
%%bash 
# pip install espnet_model_zoo
#model_name="Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best"
#espnet_model_zoo_download --unpack true --cachedir "/content/pretrained" "$model_name"
#ls -alth /content/pretrained/653d10049fdc264f694f57b49849343e/
#zipfile=pretrained/653d10049fdc264f694f57b49849343e/asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best.zip
# cp -r $zipfile  /mydrive/data/
ls -alth /mydrive/data
ZIPFILE=asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best.zip
cp /mydrive/data/$ZIPFILE ./
unzip $ZIPFILE -d pretrained_espnet

total 1.9G
-rw------- 1 root root 971M Sep 29 14:09 asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best.zip
-rw------- 1 root root  55M Sep 28 16:07 dev-clean_preprocessed.tar.gz
-rw------- 1 root root 636M Jun  7 08:35 decisions_185_StGB_no_content.jsonl.gz
-rw------- 1 root root 279M Jan  9  2020 BverfG_juris_content.jsonl.gz
Archive:  asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best.zip
 extracting: pretrained_espnet/meta.yaml  
 extracting: pretrained_espnet/exp/asr_train_asr_transformer_e18_raw_bpe_sp/config.yaml  
 extracting: pretrained_espnet/exp/lm_train_lm_adam_bpe/config.yaml  
 extracting: pretrained_espnet/exp/asr_train_asr_transformer_e18_raw_bpe_sp/54epoch.pth  
 extracting: pretrained_espnet/exp/lm_train_lm_adam_bpe/17epoch.pth  
 extracting: pretrained_espnet/exp/asr_train_asr_transformer_e18_raw_bpe_sp/RESULTS.md  
 extracting: pretrained_espnet/exp/asr_stats_raw_sp/train/feats_stats.npz  
 extracting: pretrained_espnet/data/token_list/bpe_unigram5000/bpe.m

In [7]:
%%time
%%bash
export WANDB_API_KEY=$(head -n 1 /mydrive/wandb.key | cut -d '=' -f 2 )
export WANDB_NAME="debug"
export WANDB_NOTES="just trying to track GPU usage"
export WANDB_PROJECT=espnet-asr
python -m wandb init --project $WANDB_PROJECT

rm -rf /tmp/espnet_output # start from zero, use if train-data changes somehow
#rm -rf /tmp/espnet_output/train_logs
python /mydrive/speech-recognition/espnet_main.py \
    --train_path "dev-clean_preprocessed" \
    --eval_path "dev-clean_preprocessed" \
    --pretrained_base "pretrained_espnet" \
    --num_gpus 1 \
    --batch_bins 3200000
    #  --eval_limit 40 \

Namespace(batch_bins=3200000, config_yml=None, eval_limit=None, eval_path='dev-clean_preprocessed', is_distributed=False, num_encoder_blocks=1, num_gpus=1, num_workers=1, output_path='/tmp/espnet_output', pretrained_base='pretrained_espnet', train_limit=1, train_path='dev-clean_preprocessed', vocab_size=500)



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
could not save program above cwd: /mydrive/speech-recognition/espnet_main.py
wandb: Currently logged in as: dertilo (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.5
wandb: Syncing run debug
wandb: ⭐️ View project at https://wandb.ai/dertilo/espnet-asr
wandb: 🚀 View run at https://wandb.ai/dertilo/espnet-asr/runs/15xvidiw
wandb: Run data is saved locally in wandb/run-20201014_082026-15xvidiw
wandb: Run `wandb off` to turn off syncing.
/usr/bin/python3 /mydrive/speech-recognition/espnet_main.py --train_path dev-clean_preprocessed --eval_path dev-clean_preprocessed --pretrained_base pretrained_espnet --num_gpus 1 --batch_bins 3200000
[a4f6042d69d2] 2020-10-14 08:20:28,509 (asr:281) 

CPU times: user 14.2 ms, sys: 3.89 ms, total: 18.1 ms
Wall time: 3min 17s


In [8]:
%%bash
ls -alth /tmp/espnet_output

total 24K
drwxrwxrwt 1 root root 4.0K Oct 14 08:23 ..
drwxr-xr-x 5 root root 4.0K Oct 14 08:23 train_logs
drwxr-xr-x 5 root root 4.0K Oct 14 08:21 .
drwxr-xr-x 4 root root 4.0K Oct 14 08:20 stats
drwxr-xr-x 4 root root 4.0K Oct 14 08:20 manifests
-rw-r--r-- 1 root root 1.5K Oct 14 08:20 config.yml
